In [1]:
import torch
import sklearn.datasets
import sys
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
import numpy as np
import cv2

from denoising_diffusion_pytorch import Unet, GaussianDiffusion


pio.renderers.default = "chrome"
pio.templates.default = "plotly"

NOTE: Redirects are currently not supported in Windows or MacOs.


In [2]:
world_map = cv2.imread('/Users/kodamaakira/research-embedding/notebooks/continuous_projection/colored_world_map3.jpeg')

In [2]:
dog_img = cv2.imread("/Users/kodamaakira/denoising-diffusion-pytorch/images/dog.png")

In [3]:
dog_img.shape

(128, 128, 3)

In [26]:
torch.from_numpy(np.expand_dims(np.transpose(world_map[:128, :128], (2, 0, 1)), 0)).shape

torch.Size([1, 3, 128, 128])

In [6]:
fig = px.imshow(world_map)
fig.show()

In [4]:
def normalize_img(img):
    result = []
    for channel in range(img.shape[0]):
        result.append(img[channel] / img[channel].max())

    return np.stack(result, axis=0)

In [5]:
model = Unet(
    dim=64,
    dim_mults=(1, 2, 4, 8)
)

diffusion = GaussianDiffusion(
    model,
    image_size=128,
    timesteps=1000,  # number of steps
    loss_type='l1'  # L1 or L2
)

training_images = torch.randn(8, 3, 128, 128)  # images are normalized from 0 to 1
input_img = np.stack([normalize_img(np.transpose(dog_img, (2, 0, 1)))] * 8, axis=0)

noise_sample, t = diffusion(torch.from_numpy(input_img).float())
noise_sample = noise_sample.numpy()
t

tensor([817, 497, 139, 461, 478, 710,   1, 183])

In [24]:
noise_sample.shape

torch.Size([8, 3, 128, 128])

In [21]:
normalize_img(np.transpose(world_map[:128, :128], (2, 0, 1))).shape

(1, 3, 128, 128)

In [15]:
input_img.shape

(8, 1, 128, 128)

In [23]:
fig = px.imshow(np.transpose(input_img[0], (1, 2, 0)))
fig.update_layout(title="normalized input image")
fig.show()

In [14]:
def re_normalize(noises):
    result = []
    for noise in noises:
        min_value = np.tile(noise.reshape(3, -1).min(axis=1).reshape(3, 1, 1), (1, 128, 128))
        x = noise - min_value
        result.append(x / np.tile(x.reshape(3, -1).max(axis=1).reshape(3, 1, 1), (1, 128, 128)) * 255)

    return np.stack(result, axis=0)

In [15]:
renormalized_noise_sample = re_normalize(noise_sample)

renormalized_noise_sample.shape

(8, 3, 128, 128)

In [20]:
target_id = 3
fig = px.imshow(np.transpose(renormalized_noise_sample[target_id], (1, 2, 0)))
fig.update_layout(title=f"t = {t[target_id]}")
fig.show()

In [11]:
input_img.shape

(1, 3, 128, 128)

In [10]:
fig = go.Figure()
reshaped_input = np.squeeze(input_img).reshape(3, -1).T
fig.add_trace(go.Scatter3d(
    x=reshaped_input[:, 0], y=reshaped_input[:, 1], z=reshaped_input[:, 2],
    mode="markers",
    marker={
        "size": 2
    }
))
fig.show()

In [7]:
target_id = 4
fig = go.Figure()
reshaped_noise = noise_sample.reshape(8, 3, -1)
fig.add_trace(go.Scatter3d(
    x=reshaped_noise[target_id, 0], y=reshaped_noise[target_id, 1], z=reshaped_noise[target_id, 2],
    mode="markers",
    marker={
        "size": 2
    }
))
fig.update_layout(title=f"t = {t[target_id]}")
fig.show()

In [10]:
target_id = 3
fig = go.Figure()
reshaped_noise = noise_sample.reshape(8, 3, -1)
fig.add_trace(go.Scatter(
    x=reshaped_noise[target_id, 0], y=reshaped_noise[target_id, 1],
    mode="markers",
    marker={
        "size": 4
    }
))
fig.update_layout(title=f"t = {t[target_id]}")
fig.show()

In [ ]:
np.transpose(training_images[0], (1, 2, 0))